In [1]:
import pandas as pd
import numpy as np
import re


In [109]:
def extract_key_rent_info(df_in):
    """
    接收原始的 rent DataFrame，提取并清洗指定的关键列，
    返回一个简洁的新 DataFrame。
    """
    df = df_in.copy()
    print("--- 开始提取关键租赁信息 ---")
    
    # 创建一个空的新 DataFrame 来存储结果
    df_new = pd.DataFrame()
    
    # 1. 价格 (Price)
    if 'Price' in df.columns:
        print("提取 [Price]...")
        df_new['Price_log'] = np.log1p(df['Price'])
    
    # 2. 楼层 (提取楼层类型和总楼层)
    s = df['楼层'].astype(str)
    
    # 提取总楼层数
    # 这个正则表达式会查找斜杠'/'或'共'字后面的数字
    df_new['总楼层'] = pd.to_numeric(s.str.extract(r'(?:/|共)(\d+)').squeeze(), errors='coerce')
    
    # 提取楼层类型
    # 这个正则表达式会查找开头部分的中文描述或数字
    df_new['楼层类型'] = s.str.extract(r'([^\d/]+|\d+)(?=/)').squeeze().fillna('未知')
    # 对于像 "4/6层" 这样的格式，我们进一步处理
    # 如果'楼层类型'是纯数字，就根据其与总楼层的比例来判断
    
    def classify_floor(row):
        floor_type = row['楼层类型']
        total_floors = row['总楼层']
        
        # 如果类型已经是中文（高、中、低），直接返回
        if any(keyword in str(floor_type) for keyword in ['高', '中', '低', '顶', '底']):
            return floor_type
            
        # 尝试将 floor_type 转为数字
        try:
            current_floor = int(floor_type)
            if pd.notna(total_floors) and total_floors > 0:
                ratio = current_floor / total_floors
                if ratio < 0.33:
                    return '低楼层'
                elif ratio < 0.66:
                    return '中楼层'
                else:
                    return '高楼层'
        except (ValueError, TypeError):
            pass # 如果无法转换，则保持原样或返回未知
            
        return '未知' # 默认情况

    # 应用这个分类函数
    # 注意：这需要一个临时的DataFrame来同时访问两列
    temp_df = pd.DataFrame({
        '楼层类型': df_new['楼层类型'],
        '总楼层': df_new['总楼层']
    })
    df_new['楼层类型'] = temp_df.apply(classify_floor, axis=1)

    # 3. 户型 (例如: '1室1厅1卫')
    if '户型' in df.columns:
        print("提取 [户型]...")
        df_new['户型'] = df['户型']
        # (可选) 您也可以在这里解析出卧室数、客厅数等
        # df_new['卧室数'] = df['户型'].astype(str).str.extract(r'(\d+)[室房]').fillna(0).astype(int)

    # 4. 装修
    if '装修' in df.columns:
        print("提取 [装修]...")
        df_new['装修'] = df['装修'].fillna('未装修')

    # 5. 电梯
    if '电梯' in df.columns:
        print("提取 [电梯]...")
        df_new['有无电梯'] = df['电梯'].apply(lambda x: '有' if str(x) == '有' else '无')

    # 6. 城市
    if '城市' in df.columns:
        print("提取 [城市]...")
        # 这里假设城市已经是您需要的格式，如果需要映射，可以在此添加
        df_new['城市'] = df['城市']

    if '区县' in df.columns:
        print("提取 [区县]...")
        # 这里假设城市已经是您需要的格式，如果需要映射，可以在此添加
        df_new['区县'] = df['区县']
        
    if '板块' in df.columns:
        print("提取 [板块]...")
        # 这里假设城市已经是您需要的格式，如果需要映射，可以在此添加
        df_new['板块'] = df['板块']
        
    # --- 对费用相关列进行清洗 ---
    def extract_and_average_numeric(series):
        numeric_parts = series.astype(str).str.findall(r'(\d+\.?\d*)')
        def get_average(num_list):
            if not num_list: return np.nan
            return np.mean([float(n) for n in num_list])
        return numeric_parts.apply(get_average)

    #7.面积
    extracted_series = df['面积'].str.extract(r'(\d+\.?\d*)', expand=False)
    df_new['面积'] = pd.to_numeric(extracted_series)
    df_new['面积_平方'] = df_new['面积'] ** 2

    # 8. 停车位
    if '停车位' in df.columns:
        print("提取并清洗 [停车位]...")
        df_new['停车位数量'] = extract_and_average_numeric(df['停车位'])
    
    # 9. 停车费
    if '停车费用' in df.columns:
        print("提取并清洗 [停车费用]...")
        df_new['停车月费_均值'] = extract_and_average_numeric(df['停车费用'])

    # 10. 物业费
    if '物 业 费' in df.columns:
        print("提取并清洗 [物 业 费]...")
        df_new['物业费_均值'] = extract_and_average_numeric(df['物 业 费'])

    # 11. 综合评价 (从客户反馈中提取)
    if '客户反馈' in df.columns:
        print("提取 [综合评价]...")
        positive_keywords = ['体验佳', '干净', '安静', '方便', '好', '安全', '阳光充足', '整洁']
        negative_keywords = ['老旧', '费高', '噪音', '通风差', '潮湿', '漏水', '老化', '乱', '卫生差','一般']
        df_new['综合评价得分'] = df['客户反馈'].fillna('').apply(
            lambda x: sum(1 for word in positive_keywords if word in x) - 
                      sum(1 for word in negative_keywords if word in x)
        )
     # --- 统一处理新表中的缺失值 ---
    print("填充新表中的数值型缺失值...")
    for col in df_new.select_dtypes(include=np.number).columns:
        df_new[col] = df_new[col].fillna(0) # 用0填充所有数值缺失
    
    df_new['室'] = df_new['户型'].str.extract(r'(\d+)室').fillna(0).astype(int)
    df_new['厅'] = df_new['户型'].str.extract(r'(\d+)厅').fillna(0).astype(int)
    df_new['卫'] = df_new['户型'].str.extract(r'(\d+)卫').fillna(0).astype(int)
    df_new = pd.get_dummies(df_new, columns=['楼层类型', '装修', '有无电梯','区县'], drop_first=True)   
    
   
        
    return df_new

In [110]:
    df_rent_raw = pd.read_csv('ruc_Class25Q2_train_rent.csv', low_memory=False) 
    print("成功加载租金数据！")

    # 2. 调用新的提取函数
    df_rent_key_info = extract_key_rent_info(df_rent_raw)

    # 3. 显示最终的简洁表格
    print("\n--- 提取的关键信息表 ---")
    display(df_rent_key_info.head(15))
    print("\n新表基本信息:")
    df_rent_key_info.info()


成功加载租金数据！
--- 开始提取关键租赁信息 ---
提取 [Price]...
提取 [户型]...
提取 [装修]...
提取 [电梯]...
提取 [城市]...
提取 [区县]...
提取 [板块]...
提取并清洗 [停车位]...
提取并清洗 [停车费用]...
提取并清洗 [物 业 费]...
提取 [综合评价]...
填充新表中的数值型缺失值...

--- 提取的关键信息表 ---


,Price_log,总楼层,户型,板块,面积,面积_平方,停车位数量,停车月费_均值,物业费_均值,综合评价得分,...,城市_2,城市_3,城市_4,城市_5,城市_6,城市_7,城市_8,城市_9,城市_10,城市_11
0,13.391852,6.0,1室1厅1卫,145.0,36.42,1326.4164,450.0,150.000000,1.475,1,...,False,False,False,False,False,False,False,False,False,False
1,13.408163,6.0,1室1厅1卫,581.0,41.00,1681.0000,150.0,150.000000,0.875,-1,...,False,False,False,False,False,False,False,False,False,False
2,13.564769,18.0,1室1厅1卫,355.0,37.36,1395.7696,965.0,500.000000,2.330,0,...,False,False,False,False,False,False,False,False,False,False
3,13.324628,10.0,3室1厅2卫,1102.0,55.42,3071.3764,500.0,550.000000,3.200,0,...,False,False,False,False,False,False,False,False,False,False
4,13.810230,18.0,1室1厅1卫,250.0,49.30,2430.4900,400.0,150.000000,1.000,0,...,False,False,False,False,False,False,False,False,False,False
5,13.739308,17.0,1室1厅1卫,817.0,42.00,1764.0000,0.0,150.000000,0.800,0,...,False,False,False,False,False,False,False,False,False,False
6,13.699546,17.0,1室1厅1卫,927.0,42.00,1764.0000,150.0,120.000000,1.750,1,...,False,False,False,False,False,False,False,False,False,False
7,13.679223,17.0,1室1厅1卫,817.0,42.12,1774.0944,0.0,150.000000,0.800,-1,...,False,False,False,False,False,False,False,False,False,False
8,13.763865,17.0,1室1厅1卫,927.0,42.12,1774.0944,150.0,120.000000,1.750,0,...,False,False,False,False,False,False,False,False,False,False
9,14.534337,25.0,3室2厅,367.0,176.00,30976.0000,280.0,200.666667,2.800,0,...,False,False,False,False,False,False,False,False,False,False



新表基本信息:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 98899 entries, 0 to 98898
Columns: 133 entries, Price_log to 城市_11
dtypes: bool(120), float64(8), int32(3), int64(1), object(1)
memory usage: 20.0+ MB


In [111]:
df = df_rent_key_info.drop(['户型','板块','城市'], axis=1)

In [112]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler

In [113]:
X = df.drop('Price_log', axis=1)
y = df['Price_log']

# 识别出独热编码的列 (这些列的值只有0和1)
# 我们通过检查每列的唯一值数量来判断
one_hot_cols = [col for col in X.columns if X[col].nunique() <= 2]

# 连续特征列就是除了独热编码列之外的所有列
continuous_cols = [col for col in X.columns if col not in one_hot_cols]




# --- 步骤2：创建并定义 ColumnTransformer ---

# 创建一个转换器列表
# 每个转换器是一个元组 (name, transformer, columns)
preprocessor = ColumnTransformer(
    transformers=[
        # 第一个转换器：对连续特征应用 StandardScaler
        ('num', StandardScaler(), continuous_cols),
        
        # 第二个转换器：对独热编码特征不做任何处理
        # 'passthrough' 是一个特殊的字符串，表示“保持这些列不变”
        ('cat', 'passthrough', one_hot_cols)
    ],
    remainder='drop' # 如果有未指定的列，可以选择'drop'或'passthrough'
)


# --- 步骤3：应用转换器 ---

# 使用 preprocessor 对特征矩阵 X 进行转换
# .fit_transform() 会返回一个 NumPy 数组
X_processed = preprocessor.fit_transform(X)

# --- （可选）将结果转回 DataFrame 以便查看 ---
# ColumnTransformer 会改变列的顺序 (它会把转换后的列按顺序拼接)
# 所以我们需要重新构建列名列表
new_column_order = continuous_cols + one_hot_cols
X_processed_df = pd.DataFrame(X_processed, columns=new_column_order, index=X.index)

print("\n--- 预处理后的特征数据 (前5行) ---")
print("注意：只有连续特征被标准化了，独热编码列保持0/1不变")
print(X_processed_df.head())


--- 预处理后的特征数据 (前5行) ---
注意：只有连续特征被标准化了，独热编码列保持0/1不变
        总楼层        面积     面积_平方     停车位数量   停车月费_均值    物业费_均值    综合评价得分  \
0 -1.242860 -1.033841 -0.642452 -0.342893 -0.316011 -0.200615  1.210083   
1 -1.242860 -0.926439 -0.609785 -0.546947 -0.316011 -0.366209 -1.439169   
2 -0.162895 -1.011798 -0.636063  0.007398  1.024192  0.035356 -0.114543   
3 -0.882872 -0.588285 -0.481689 -0.308884  1.215649  0.275468 -0.114543   
4 -0.162895 -0.731801 -0.540734 -0.376902 -0.316011 -0.331710 -0.114543   

          室         厅         卫  ...  城市_2  城市_3  城市_4  城市_5  城市_6  城市_7  \
0 -1.058631 -0.436462  0.806635  ...   0.0   0.0   0.0   0.0   0.0   0.0   
1 -1.058631 -0.436462  0.806635  ...   0.0   0.0   0.0   0.0   0.0   0.0   
2 -1.058631 -0.436462  0.806635  ...   0.0   0.0   0.0   0.0   0.0   0.0   
3  0.793800 -0.436462  2.263187  ...   0.0   0.0   0.0   0.0   0.0   0.0   
4 -1.058631 -0.436462  0.806635  ...   0.0   0.0   0.0   0.0   0.0   0.0   

   城市_8  城市_9  城市_10  城市_11  
0   0.0  

In [114]:
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV
from sklearn.linear_model import LinearRegression, Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.metrics import r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

In [115]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold # 导入 KFold
np.random.seed(42)

In [116]:
target_column = 'Price_log'
X_raw = df.drop(target_column, axis=1) # X_raw 应该是除目标列外的所有列

# y_log 直接就是 df 中的 'Price_log' 列，无需再次转换
y_log = df[target_column] # <--- 正确的做法

X_processed = pd.get_dummies(X_raw, drop_first=True)
df_processed = pd.concat([X_processed, y_log], axis=1)

# 1c. 移除异常值 (在处理过的特征和对数化的目标上进行)
Q1 = df_processed[target_column].quantile(0.25)
Q3 = df_processed[target_column].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR
df_cleaned = df_processed[(df_processed[target_column] >= lower_bound) & (df_processed[target_column] <= upper_bound)]

print(f"移除异常值后的数据集形状: {df_cleaned.shape}")
print(f"用于预测的总样本数量: {df_cleaned.shape[0]}")

# 1d. 定义最终的完整数据集 (X_full, y_full) 和训练/测试集
X_full = df_cleaned.drop(target_column, axis=1)
y_full = df_cleaned[target_column]
X_train, X_test, y_train, y_test = train_test_split(X_full, y_full, test_size=0.2, random_state=42)
print("-" * 50)

# --- 第2步：定义模型 Pipelines ---
# 使用 Pipeline 将标准化和模型训练打包
print("步骤2: 定义模型 Pipelines...")

ols_stabilized_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', Ridge(alpha=1e-8)) 
])

# RidgeCV 和 LassoCV 自带交叉验证来寻找最佳alpha
ridge_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', RidgeCV(alphas=np.logspace(-4, 4, 100))) # 提供一个alpha范围供其搜索
])

lasso_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('model', LassoCV(cv=6, max_iter=10000, random_state=42)) # cv=6与外部验证折数保持一致
])

models = {
    'OLS_stabilized': ols_stabilized_pipeline, 
    'Lasso': lasso_pipeline, 
    'Ridge': ridge_pipeline
}
print("Pipelines 定义完成。")
print("-" * 50)




移除异常值后的数据集形状: (98205, 131)
用于预测的总样本数量: 98205
--------------------------------------------------
步骤2: 定义模型 Pipelines...
Pipelines 定义完成。
--------------------------------------------------


In [117]:
results_list = [] 
def mae_original_scale_scorer(estimator, X, y_log):
    y_pred_log = estimator.predict(X)
    # 1. 将预测值和真实值转换回原始尺度
    y_pred = np.expm1(y_pred_log)
    y_true = np.expm1(y_log)
    # 2. 在原始尺度上计算 MAE
    return mean_absolute_error(y_true, y_pred)

for name, model_pipeline in models.items():
    print(f"--- 正在评估模型: {name} ---")
    
    # a) 样本内 (In-sample) 性能
    # 在完整的清洗后数据集上训练
    model_pipeline.fit(X_full, y_full)
    y_pred_in_sample_log = model_pipeline.predict(X_full)
    # 将预测值和真实值转换回原始尺度进行评估
    y_pred_in_sample = np.expm1(y_pred_in_sample_log)
    y_true_in_sample = np.expm1(y_full)
    in_sample_mae = mean_absolute_error(y_true_in_sample, y_pred_in_sample)


    # b) 样本外 (Out-of-sample) 性能
    # 在训练集上训练，在测试集上评估
    model_pipeline.fit(X_train, y_train)
    y_pred_out_sample_log = model_pipeline.predict(X_test)
    y_pred_out_sample = np.expm1(y_pred_out_sample_log)
    y_true_out_sample = np.expm1(y_test)
    out_sample_mae = mean_absolute_error(y_true_out_sample, y_pred_out_sample)

    kfold_splitter = KFold(n_splits=6, shuffle=True, random_state=42)
    # c) 6折交叉验证 (Cross-validation) 性能
    # 使用我们新定义的 mae_original_scale_scorer
    cv_mae_scores = cross_val_score(model_pipeline, X_full, y_full, cv=kfold_splitter, scoring=mae_original_scale_scorer)
    cv_mae = cv_mae_scores.mean()

    # 存储结果
    results_list.append({
        'Model': name,
        'In-sample MAE': f"{in_sample_mae:,.2f}", #  格式化为千分位
        'Out-of-sample MAE': f"{out_sample_mae:,.2f}",
        'Cross-validation MAE': f"{cv_mae:,.2f}",
    })

# --- 第4步：展示结果 ---
print("-" * 50)
print("步骤4: 展示最终评估结果 (原始尺度 MAE)...")
results_df = pd.DataFrame(results_list).set_index('Model')
print(results_df)

--- 正在评估模型: OLS_stabilized ---
--- 正在评估模型: Lasso ---


KeyboardInterrupt: 

In [91]:
    df_rent_test_raw = pd.read_csv('ruc_Class25Q2_test_rent.csv', low_memory=False) 
    print("成功加载租金数据！")

成功加载租金数据！


In [92]:
df_test_raw = df_rent_test_raw.copy()
test_ids = df_test_raw['ID'] if 'ID' in df_test_raw.columns else df_test_raw.index

print(f"成功加载测试数据！形状为: {df_test_raw.shape}")

# 2. 对测试数据应用【完全相同】的预处理函数
if 'Price' not in df_test_raw.columns:
    df_test_raw['Price'] = 0 # 添加一个值为0的假列

df_test_processed = extract_key_rent_info(df_test_raw)
X_test_final = df_test_processed.drop(['户型', 'Price_log'], axis=1, errors='ignore')

# 3. 【极其重要】确保测试集的特征列与训练集的特征列完全对齐
print("正在对齐测试集和训练集的特征列...")
X_test_aligned = X_test_final.reindex(columns=X_full.columns, fill_value=0)
print("测试数据预处理和对齐完成！")
print("-" * 50)

# --- 步骤6：循环遍历所有模型，进行预测并保存文件 ---

print("步骤6: 开始为每个模型生成预测文件...")

for name, model_pipeline in models.items():
    print(f"\n--- 正在使用模型 '{name}' 进行预测 ---")
    
    # 1. 使用已经训练好的模型进行预测 (无需重新fit)
    # Pipeline 会自动对 X_test_aligned 应用 scaler
    predictions_log = model_pipeline.predict(X_test_aligned)
    
    # 2. 将预测结果从对数尺度转换回原始价格尺度
    predictions_original_scale = np.expm1(predictions_log)
    
    # 3. 创建用于提交的 DataFrame
    submission_df = pd.DataFrame({
        'ID': test_ids,
        'Price': predictions_original_scale
    })
    
    # 4. 格式化并保存为 CSV 文件
    submission_df['Price'] = submission_df['Price'].astype(int)
    
    # 使用模型名称创建唯一的文件名
    filename = f'submission_{name}.csv'
    submission_df.to_csv(filename, index=False)
    
    print(f"成功生成提交文件: '{filename}'")
    print("文件内容预览:")
    print(submission_df.head())

print("\n--------------------------------------------------")
print("所有模型的预测文件均已生成完毕！")

成功加载测试数据！形状为: (9773, 46)
--- 开始提取关键租赁信息 ---
提取 [Price]...
提取 [户型]...
提取 [装修]...
提取 [电梯]...
提取 [城市]...
提取 [区县]...
提取 [板块]...
提取并清洗 [停车位]...
提取并清洗 [停车费用]...
提取并清洗 [物 业 费]...
提取 [综合评价]...
填充新表中的数值型缺失值...
正在对齐测试集和训练集的特征列...
测试数据预处理和对齐完成！
--------------------------------------------------
步骤6: 开始为每个模型生成预测文件...

--- 正在使用模型 'OLS_stabilized' 进行预测 ---
成功生成提交文件: 'submission_OLS_stabilized.csv'
文件内容预览:
        ID    Price
0  2000000   219999
1  2000001   485018
2  2000002   431376
3  2000003  1505686
4  2000004   742607

--- 正在使用模型 'Lasso' 进行预测 ---
成功生成提交文件: 'submission_Lasso.csv'
文件内容预览:
        ID    Price
0  2000000   220037
1  2000001   485427
2  2000002   430432
3  2000003  1498793
4  2000004   741946

--- 正在使用模型 'Ridge' 进行预测 ---
成功生成提交文件: 'submission_Ridge.csv'
文件内容预览:
        ID    Price
0  2000000   220003
1  2000001   485024
2  2000002   431377
3  2000003  1505631
4  2000004   742605

--------------------------------------------------
所有模型的预测文件均已生成完毕！


In [118]:
df_part1=pd.read_csv('price prediction.csv') 
df_part2=pd.read_csv('submission_Ridge.csv') 
df_full = pd.concat([df_part1, df_part2],ignore_index=True)
df_full.to_csv('prediction.csv',index=False)